In [1]:
from face2voice.models.SpeakerEncoder import SpeakerEncoder
from torchvision.transforms import ToTensor
from openvoice.api import ToneColorConverter
import torch
from openvoice import se_extractor


/Users/mcmk/projects/face2voice/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
tone_color_converter = ToneColorConverter(f'checkpoints/tone_conv/config.json', device="cpu")
tone_color_converter.load_ckpt(f'checkpoints/tone_conv/checkpoint.pth')

/Users/mcmk/projects/face2voice/.venv/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'checkpoints/tone_conv/checkpoint.pth'
missing/unexpected keys: [] []


In [3]:
speaker_encoder = SpeakerEncoder(config_path="checkpoints/tone_conv/config.json", ckpt_path="checkpoints/tone_conv/checkpoint.pth")

Loaded checkpoint 'checkpoints/tone_conv/checkpoint.pth'
missing/unexpected keys: [] []


In [ ]:
tgt_emb = speaker_encoder.encode_single(audio="outputs/example_reference_v2_rHQICiKpUG3_^AcmH/spec.pth", input="spec_tensor", return_numpy=False)

tensor(-0.0492)

In [7]:
src_emb = speaker_encoder.encode_single(audio="outputs/demo_speaker0_v2_L54YAlaNxwu9WlYU/spec.pth", input="spec_tensor", return_numpy=False)

In [10]:
tgt_emb = torch.tensor(tgt_emb).transpose(1, 2)
src_emb = torch.tensor(src_emb).transpose(1, 2)
print(type(tgt_emb), type(src_emb))

<class 'torch.Tensor'> <class 'torch.Tensor'>


/var/folders/nw/7l6j9hqx7_3gyljpmz012p400000gn/T/ipykernel_43860/1530710759.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_emb = torch.tensor(tgt_emb).transpose(1, 2)
/var/folders/nw/7l6j9hqx7_3gyljpmz012p400000gn/T/ipykernel_43860/1530710759.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_emb = torch.tensor(src_emb).transpose(1, 2)


In [11]:
speaker_encoder.tone_color_converter.convert(audio_src_path="resources/example_reference.mp3", src_se=src_emb, tgt_se=tgt_emb, output_path="outputs/testwspec2.wav")